In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [5]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
df.duplicated().sum()

418

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.duplicated().sum()

0

## Remove Tags

In [10]:
import re

def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'),'',raw_text)
    return cleaned_text

In [11]:
df['review'] = df['review'].apply(remove_tags)

In [12]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [13]:
df['review'][1]

'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well done.'

## Lowercase

In [14]:
df['review'] = df['review'].apply(lambda x:x.lower())

## Remove Stopwords

In [15]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

## Lemmatization

In [16]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = word_tokenize(text.lower())
    return " ".join([lemmatizer.lemmatize(w) for w in words if w.isalpha()])

# Apply lemmatization and overwrite df['review']
df["review"] = df["review"].apply(lemmatize_text)

In [17]:
df

,review,sentiment
0,one reviewer mentioned watching oz episode hoo...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there family little boy jake think t...,negative
4,petter mattei love time money visually stunnin...,positive
...,...,...
49995,thought movie right good job creative original...,positive
49996,bad plot bad dialogue bad acting idiotic direc...,negative
49997,catholic taught parochial elementary school nu...,negative
49998,going disagree previous comment side maltin on...,negative


In [18]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [19]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [20]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2)

In [21]:
model.build_vocab(story)

In [22]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(27406244, 28986175)

In [23]:
len(model.wv.index_to_key)

53525

In [24]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [25]:
document_vector(df['review'].values[0])

array([ 0.07068185,  0.00389054, -0.5996048 ,  0.25753272, -0.07404784,
       -0.55984384, -0.01705534,  0.19954829,  0.02257606, -0.5022294 ,
        0.05050943,  0.01266519,  0.3988665 , -0.11594416,  0.11004079,
       -0.04099844,  0.63226885, -0.327192  ,  0.02860288, -0.12574382,
        0.14813027,  0.56690705,  0.12778442, -0.17519158,  0.0809336 ,
        0.30175868,  0.36616796,  0.0600568 , -0.08377405, -0.05252787,
        0.46757245, -0.20891398,  0.42213827, -0.40865126,  0.12045193,
       -0.1303101 ,  0.362845  ,  0.0866489 , -0.19851749, -0.56395435,
        0.02133175, -0.5915511 ,  0.341293  , -0.16800375, -0.04987598,
        0.03239889,  0.01751792, -0.4632213 , -0.00824806, -0.6234907 ,
       -0.2406076 ,  0.09022981,  0.23045103,  0.44557095,  0.05379141,
        0.54044515,  0.11369697,  0.52514446,  0.03254578,  0.67927694,
        0.10779023,  0.21629047,  0.3166311 ,  0.11571511, -0.11553985,
        0.22780968, -0.26498073,  0.1778207 , -0.8161149 ,  0.54

In [26]:
from tqdm import tqdm

X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|████████████████████████████████████████████████████████████████████████████| 49582/49582 [05:09<00:00, 160.44it/s]


In [27]:
X = np.array(X)

In [29]:
X[0]

array([ 0.07068185,  0.00389054, -0.5996048 ,  0.25753272, -0.07404784,
       -0.55984384, -0.01705534,  0.19954829,  0.02257606, -0.5022294 ,
        0.05050943,  0.01266519,  0.3988665 , -0.11594416,  0.11004079,
       -0.04099844,  0.63226885, -0.327192  ,  0.02860288, -0.12574382,
        0.14813027,  0.56690705,  0.12778442, -0.17519158,  0.0809336 ,
        0.30175868,  0.36616796,  0.0600568 , -0.08377405, -0.05252787,
        0.46757245, -0.20891398,  0.42213827, -0.40865126,  0.12045193,
       -0.1303101 ,  0.362845  ,  0.0866489 , -0.19851749, -0.56395435,
        0.02133175, -0.5915511 ,  0.341293  , -0.16800375, -0.04987598,
        0.03239889,  0.01751792, -0.4632213 , -0.00824806, -0.6234907 ,
       -0.2406076 ,  0.09022981,  0.23045103,  0.44557095,  0.05379141,
        0.54044515,  0.11369697,  0.52514446,  0.03254578,  0.67927694,
        0.10779023,  0.21629047,  0.3166311 ,  0.11571511, -0.11553985,
        0.22780968, -0.26498073,  0.1778207 , -0.8161149 ,  0.54

In [30]:
X.shape

(49582, 100)

In [31]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(df['sentiment'])

In [32]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [33]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.8406776242815368